 # EXAMEN GRANDS SYSTEMES - 4 GMM

Importation des modules de calculs scientifiques.

In [ ]:
import numpy as np
import numpy.linalg as npl
import scipy as sp
import scipy.sparse as sps
import scipy.optimize as spo
import matplotlib.pyplot as plt
%matplotlib inline

### Instructions pour cet examen

- Les documents et les objets connectés sont strictement interdits. 
- Sauf contre-indications, il ne faut pas utiliser de fonctions des bibliothèques Python lorsque votre propre implémentation est demandée. 
- Lors de la correction, vos fonctions seront testées avec différents arguments, il est donc **primordial** de respecter la syntaxe et le format des fonctions.
- Les matrices et vecteurs sont pris au format `array` de numpy/scipy. De plus, les matrices peuvent être au format sparse.
- Vous êtes libre de définir des fonctions autres que celles que l'on vous demande explicitement afin de rendre vos codes plus simples ou plus lisibles.

## I. Calcul d'éléments propres par la minimisation de Shmoukly

Soit $A\in M_n(\mathbb{R})$ une matrice symétrique définie positive. Pour tout $m\in\mathbb{R}$, on considère à présent la fonctionnelle $J_m$ donnée, $\forall x\in\mathbb{R}^n$, par

$$
J_m(x) = \frac12\langle x,Ax\rangle - m \|x\|_2.
$$

On remarque que le gradient de $J$ se calcule comme

$$
\nabla J_m(x) = Ax - m \frac{x}{\|x\|_2}.
$$

Le but de cette partie est de minimiser cette fonctionnelle. En effet, l'unique minimum de $J$ vérifie les conditions d'optimalité du premier ordre, c'est-à-dire

$$
Ax - m \frac{x}{\|x\|_2} = 0 \quad \Leftrightarrow \quad Ax = \frac{m}{\|x\|_2} x,
$$

ce qui nous donne donc l'élément propre $(x,m/\|x\|_2)$. Il reste à voir ce que cela donne en pratique...

Afin de calculer le minimum de cette fonction, nous allons utiliser des méthodes de type gradient. C'est-à-dire que l'on va construire une suite $(x_k)_{k\geq 0}$ à partir d'une relation itérative du type

$$
x_{k+1} = x_k + \alpha_k p_k,
$$

où $p_k$ est une direction de descente et $\alpha_k$ un scalaire correspondant au pas de la méthode. 

### 1) Algorithme de gradient classique

On commence par un algorithme de gradient classique prenant $p_k = -\nabla J_m(x_k)$ et $\alpha_k = h\in\mathbb{R}$.
L'algorithme prend la forme suivante

$$
\left\{\begin{array}{ll}
\textrm{Choisir }x_0\in\mathbb{R}^n
\\ \textrm{Pour }k = 0,1,2,\ldots,\textrm{itermax}:
\\ \hspace{2em} p_k = -\nabla J_m(x_k)
\\ \hspace{2em} x_{k+1} = x_k + h p_k
\\ \hspace{2em} \textrm{Si }\|h p_k\|_2 < \varepsilon \|x_k\|_2 :
\\ \hspace{4em} \textrm{Fin de la boucle sur }k
\end{array}\right.
$$

>**A faire** : Implémenter la fonction `Gradient_Cla_J` qui met en oeuvre l'algorithme précédent. Elle prendra en arguments d'entrée:
- une matrice carré `A` de taille $n$,
- un scalaire `m` correspondant à $m$,
- un scalaire `h` correspondant au pas $h$,
- un vecteur initial `x_0` de taille $n$,
- une tolérance `epsilon` (optionel à `1e-15`),
- un nombre maximal d'itérations `itermax` (optionel à `1e4`),

>et donnera en sortie:
- le vecteur `x` solution de la minimisation de la fonction $J_m$,
- le nombre d'itérations réalisé par la méthode.

In [ ]:
# VOTRE CODE ICI

>**A faire** : Tester votre fonction avec le script ci-dessous.

In [ ]:
m = 1
h = 0.5
x_0 = np.random.random(3)
A = np.array([[1.,1.,0.],[1.,2.,0.],[0.,0.,3.]])
x,k = Gradient_Cla_J(A,m,h,x_0)
print("Nombre d'itérations: ",k)
print("Quotient de Rayleigh de x: ",np.dot(np.dot(A,x),x)/npl.norm(x)**2)
print("Valeurs propres de A: ",npl.eig(A)[0])

**Question 1**: A priori, vers quelle élément propre converge la méthode?

Votre réponse ici

>**A faire** : Tester votre fonction avec le script ci-dessous.

In [ ]:
L = np.zeros(100)
I = np.zeros(100)
h = 0.5
x_0 = np.random.random(3)
A = np.array([[1.,1.,0.],[1.,2.,0.],[0.,0.,3.]])
for k in range(100):
    x,niter = Gradient_Cla_J(A,k+1,h,x_0)
    I[k] = niter
    L[k] = np.dot(np.dot(A,x),x)/npl.norm(x)**2
plt.plot(L)
plt.show()
plt.plot(I)
print("Nombre d'itérations: ",k)
print("Quotient de Rayleigh de x: ",)

## 2) Algorithme de gradient de plus profonde descente

On passe maintenant à un algorithme de gradient de la plus profonde descente en prenant $p_k = -\nabla J_m(x_k)$ et $\alpha_k$ solution du problème de minimisation

$$
\alpha_k = \textrm{argmin}_{\alpha\in\mathbb{R}} J_m(x_k + \alpha p_k).
$$

Dans ce cas, il n'y a malheureusement pas d'expression explicite pour $\alpha_k$ et on utilisera la fonction de minimisation `spo.minimize` pour l'obtenir. Cette fonction prend en argument d'entrée:
- une fonction `f` dont on cherche le minimum,
- un point `x_0` initial.

Le script ci-dessous permet de comprendre le fonctionnement de l'argument de sortie.

In [ ]:
def g(z,k):
    return k*z**2 - z + 3
f = lambda z: g(z,1)
min = spo.minimize_scalar(f).x
print(min)

Au final, l'algorithme s'écrit donc
$$
\left\{\begin{array}{ll}
\textrm{Choisir }x_0\in\mathbb{R}^n
\\ \textrm{Pour }k = 0,1,2,\ldots,\textrm{itermax}:
\\ \hspace{2em} p_k = -\nabla J_m(x_k)
\\ \hspace{2em} \textrm{Si }\|p_k\|_2 < \varepsilon :
\\ \hspace{4em} \textrm{Fin de la boucle sur }k
\\ \hspace{2em} \alpha_k = \min_{\alpha\in\mathbb{R}} J_m(x_k + \alpha p_k)
\\ \hspace{2em} x_{k+1} = x_k + \alpha_k p_k
\\ \hspace{2em} \textrm{Si }\|\alpha_k p_k\|_2 < \varepsilon \|x_k\|_2 :
\\ \hspace{4em} \textrm{Fin de la boucle sur }k
\end{array}\right.
$$

>**A faire** : Implémenter la fonction `Gradient_Opt_J` qui met en oeuvre l'algorithme précédent. Elle prendra en arguments d'entrée:
- une matrice carré `A` de taille $n$,
- un scalaire `m` correspondant à $m$,
- un vecteur initial `x_0` de taille $n$,
- une tolérance `epsilon` (optionel à `1e-15`),
- un nombre maximal d'itérations `itermax` (optionel à `1e4`),

>et donnera en sortie:
- le vecteur `x` solution de la minimisation de la fonction $J_m$,
- le nombre d'itérations réalisé par la méthode.

In [ ]:
# VOTRE CODE ICI

>**A faire** : Tester votre fonction avec le script ci-dessous.

In [ ]:
m = 1
x_0 = np.random.random(3)
A = np.array([[1.,1.,0.],[1.,2.,0.],[0.,0.,3.]])
x,k = Gradient_Opt_J(A,m,x_0)
print("Nombre d'itérations: ",k)
print("Quotient de Rayleigh de x: ",np.dot(np.dot(A,x),x)/npl.norm(x)**2)
print("Valeurs propres de A: ",npl.eig(A)[0])

**Question 2**: D'après vos tests, quelle est la méthode la plus rapide, en terme d'itérations, entre la méthode du gradient classique et celui de plus profonde descente?

Votre réponse ici

## II. Calcul d'éléments propres par la minimisation du quotient de Rayleigh

Le but de cette partie est de reprendre les méthodes précédentes mais en les appliquant à la minimisation du quotient de Rayleigh. On rappelle que, pour une matrice $A$ carrée de taille $n\in\mathbb{N}$, le quotient de Rayleigh est définit, $\forall x\in\mathbb{R}^n$, comme

$$
r_A(x) = \frac{\langle x, Ax\rangle}{\|x\|^2_2}.
$$

Lorsque la matrice $A$ est diagonalisable et que l'on considère un élément propre $(u,\lambda)$, où $u$ est un vecteur propre associé à la valeur propre $\lambda$, on a en particulier que

$$
r_A(u) = \lambda.
$$

**Question 1**: Pour une matrice $A$ symétrique, selon vous, quelle est la solution du problème de minimisation suivant

$$
\min_{x\in\mathbb{R}^n, x\neq 0} r_A(x).
$$

Votre réponse ici

Comme pour la partie précédente, on va s'orienter vers des méthodes de type gradient afin de résoudre ce problème de minimisation. L'approche générique est de considérer une suite $(x_k)_{k\geq 0}$ construite à partir d'une relation itérative du type

$$
x_{k+1} = x_k + \alpha_k p_k,
$$

où $p_k$ est une direction de descente et $\alpha_k$ un scalaire permettant d'assurer une descente optimale. Dans cette partie, nous n'utiliserons pas la fonction `spo.minimize` puisque la recherche du $\alpha_k$ optimal s'écrit simplement comme la recherche de la racine positive du polynôme suivant

$$
Q_k(\alpha) = a_k\alpha^2 + b_k \alpha + c_k,
$$
avec
$$
\left\{\begin{array}{ll}
a_k = \langle p_k\,,\,Ap_k\rangle\langle p_k\,,\,x_k\rangle - \langle x_k\,,\,Ap_k\rangle\langle p_k\,,\,p_k\rangle
\\ b_k = \langle x_k\,,\,Ax_k\rangle\langle p_k\,,\,p_k\rangle - \langle p_k\,,\,Ap_k\rangle\langle x_k\,,\,x_k\rangle
\\ c_k = \langle x_k\,,\,Ap_k\rangle\langle x_k\,,\,x_k\rangle - \langle x_k\,,\,Ax_k\rangle\langle x_k\,,\,p_k\rangle
\end{array}\right. .
$$

### 1) Algorithme de gradient 

Le gradient du quotient de Rayleigh est donné par

$$
\nabla r_A(x) = 2\frac{Ax - r_A(x)x}{\|x\|_2^2}.
$$

On commence par un algorithme de gradient de la plus profonde descente en prenant $p_k = -\nabla r_A(x_k)$.
L'algorithme prend alors la forme suivante

$$
\left\{\begin{array}{ll}
\textrm{Choisir }x_0\in\mathbb{R}^n
\\ \textrm{Pour }k = 0,1,2,\ldots,\textrm{itermax}:
\\ \hspace{2em} p_k = -\nabla r_A(x_k)
\\ \hspace{2em} \textrm{Si }\|p_k\|_2 < \varepsilon :
\\ \hspace{4em} \textrm{Fin de la boucle sur }k
\\ \hspace{2em} \alpha_k = \min_{\alpha\in\mathbb{R}} r_A(x_k + \alpha p_k)
\\ \hspace{2em} x_{k+1} = x_k + \alpha_k p_k
\\ \hspace{2em} \textrm{Si }\|\alpha_k p_k\|_2 < \varepsilon \|x_k\|_2 :
\\ \hspace{4em} \textrm{Fin de la boucle sur }k
\end{array}\right.
$$

>**A faire** : Implémenter la fonction `Gradient_Opt_R` qui met en oeuvre l'algorithme précédent. Elle prendra en arguments d'entrée:
- une matrice carré `A` de taille $n$,
- un vecteur initial `x_0` de taille $n$,
- une tolérance `epsilon` (optionel à `1e-15`),
- un nombre maximal d'itérations `itermax` (optionel à `1e4`),

>et donnera en sortie:
- le vecteur `x` solution de la minimisation du quotient de Rayleigh,
- le nombre d'itérations réalisé par la méthode.

In [ ]:
# VOTRE CODE ICI

>**A faire** : Tester votre fonction avec le script ci-dessous.

In [ ]:
x_0 = np.array([3.,-1.,1.])
A = np.array([[1.,1.,0.],[1.,2.,0.],[0.,0.,3.]])
x,k = Gradient_Opt_R(A,x_0)
print("Nombre d'itérations: ",k)
print("Quotient de Rayleigh de x: ",np.dot(np.dot(A,x),x)/npl.norm(x)**2)
print("Valeurs propres de A: ",npl.eig(A)[0])

**Question 2**: Est-ce que la méthode donne le bon résultat?

Votre réponse ici

>**A faire** : Tester votre fonction avec le script ci-dessous. Ce script donne la matrice du laplacien pour une discrétisation par différences finies d'ordre 2.

In [ ]:
def Laplacian(n) :
    return (n+1)**2*sps.diags([2*np.ones(n),-1*np.ones(n-1),-1*np.ones(n-1)], [0, -1, 1]).tocsc()
def Eig_Lapl(n):
    return np.array(4*((n+1)*np.sin(np.arange(1,n+1)*np.pi/(2*(n+1))))**2)

N = 200
A = Laplacian(N)
x_0 = np.ones((N,1))
x,k = Gradient_Opt_R(A,x_0)
print("Nombre d'itérations: ",k)
print("Quotient de Rayleigh de x: ",(A.dot(x).T.dot(x)/npl.norm(x)**2)[0][0])
print("Première valeur propre de A: ",Eig_Lapl(N)[0])

**Question 3**: Selon vous, la précision obtenue est-elle en adéquation avec la tolérance de l'algorithme?

Votre réponse ici

### 2) Algorithme du gradient conjugué

Nous allons à présent nous tourner vers un algorithme plus sophistiqué: l'algorithme du gradient conjugé non-linéaire. La direction de descente $p_k$ est alors donnée par une famille $A$-conjuguée. Plus précisément, on prend
$$
p_k = g_k + \beta_{k-1} p_{k-1},
$$
où $g_k = - \nabla r_A(x)$ et le paramètre $\beta_{k-1}$ est donné par
$$
\beta_{k-1} = \frac{\langle g_{k}\,,\, g_k-g_{k-1} \rangle}{\langle g_{k-1}\,,\, g_{k-1} \rangle}.
$$
Ceci nous amène donc à la forme suivante de l'algorithme
L'algorithme prend alors la forme suivante

$$
\left\{\begin{array}{ll}
\textrm{Choisir }x_0\in\mathbb{R}^n
\\ \textrm{Poser }g_{-1} = -\nabla r_A(x_0)\quad\textrm{et}\quad p_{-1} = 0
\\ \textrm{Pour }k = 0,1,2,\ldots,\textrm{itermax}:
\\ \hspace{2em} g_k = -\nabla r_A(x_k)
\\ \hspace{2em} \beta_{k-1} =  \frac{\langle g_{k}\,,\, g_k - g_{k-1}\, \rangle}{\langle g_{k-1}\,,\, g_{k-1}\, \rangle}
\\ \hspace{2em} p_k =  g_k + \beta_{k-1} p_{k-1}
\\ \hspace{2em} \textrm{Si }\|p_k\|_2 < \varepsilon:
\\ \hspace{4em} \textrm{Fin de la boucle sur }k
\\ \hspace{2em} \alpha_k = \min_{\alpha\in\mathbb{R}} r_A(x_k + \alpha p_k)
\\ \hspace{2em} x_{k+1} = x_k + \alpha_k p_k
\\ \hspace{2em} \textrm{Si }\|\alpha_k p_k\|_2 < \varepsilon \|x_k\|_2 :
\\ \hspace{4em} \textrm{Fin de la boucle sur }k
\end{array}\right.
$$

>**A faire** : Implémenter la fonction `Gradient_Conj_R` qui met en oeuvre l'algorithme précédent. Elle prendra en arguments d'entrée:
- une matrice carré `A` de taille $n$,
- un vecteur initial `x_0` de taille $n$,
- une tolérance `epsilon` (optionel à `1e-15`),
- un nombre maximal d'itérations `itermax` (optionel à `1e4`),

>et donnera en sortie:
- le vecteur `x` solution de la minimisation du quotient de Rayleigh,
- le nombre d'itérations réalisé par la méthode.

In [ ]:
# VOTRE CODE ICI

>**A faire** : Tester votre fonction avec le script ci-dessous.

In [ ]:
N = 1000
A = Laplacian(N)
x_0 = np.ones((N,1))
x,k = Gradient_Conj_R(A,x_0)
print("Nombre d'itérations: ",k)
print("Quotient de Rayleigh de x: ",(A.dot(x).T.dot(x)/npl.norm(x)**2)[0][0])
print("Première valeur propre de A: ",Eig_Lapl(N)[0])

**Question 4**: Selon vous, la précision obtenue est-elle en adéquation avec la tolérance de l'algorithme?

Votre réponse ici